In [1]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from tqdm import tqdm

In [2]:
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

import time
import pandas as pd
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
import chromedriver_autoinstaller

print(selenium.__version__)

4.3.0


In [3]:
# setup chrome options
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') # ensure GUI is off
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

# set path to chromedriver as per your configuration
chromedriver_autoinstaller.install()

# set up the webdriver
driver = webdriver.Chrome(options=chrome_options)
print(driver)

<selenium.webdriver.chrome.webdriver.WebDriver (session="3bf5855067d2f81f903477f7a9348930")>


In [135]:
url = "https://www.tworld.co.kr/web/support/store/map"
driver.get(url)
time.sleep(3)
place_names = []
addresses = []
phones = []

In [136]:
element1 = driver.find_element(By.XPATH, '//*[@id="content"]/div[1]/div/div[2]/div[2]/div[1]/div[1]/div/div[1]/div[1]/div/a')
element1.click()
time.sleep(1)

element2 = driver.find_element(By.XPATH, '//*[@id="content"]/div[1]/div/div[2]/div[2]/div[1]/div[1]/div/div[1]/div[1]/div/div/div/div/ul/li[2]/a')
element2.click()
time.sleep(1)

search_input = driver.find_element(By.XPATH, '//*[@id="searchText"]')
search_input.send_keys('전북')
time.sleep(1)

search_button = driver.find_element(By.XPATH, '//*[@id="searchBtn"]')
search_button.click()
time.sleep(3)

In [137]:
for page in tqdm(range(1, 59)):
        for i in range(2):
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
                time.sleep(3)
        for i in range(1, 26):
                title_button = driver.find_element(By.XPATH, f"/html/body/div[2]/div[3]/div/div/div[1]/div/div[2]/div[2]/div[1]/div[2]/div[1]/div[{i}]/div[1]/div/a")
                place_names.append(title_button.text)

                address_element = driver.find_element(By.XPATH, f'//*[@id="store-map-layout"]/div[1]/div[{i}]/div[1]/div/div/div[3]/div')
                addresses.append(address_element.text)

                phone_element = driver.find_element(By.XPATH, f'//*[@id="store-map-layout"]/div[1]/div[{i}]/div[1]/div/div/div[4]').text
                phones.append(phone_element)

        if page < 58:

                for _ in range(2):
                        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_UP)
                        time.sleep(2)
                wait = WebDriverWait(driver, 10)
                next_button = wait.until(EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/div[3]/div/div/div[1]/div/div[2]/div[2]/div[1]/div[2]/div[2]/button[3]')))
                next_button.click()
                time.sleep(3)
                driver.execute_script("window.scrollTo(0, 0);")
                time.sleep(3)

  5%|████▎                                                                              | 3/58 [00:58<17:54, 19.54s/it]


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[2]/div[3]/div/div/div[1]/div/div[2]/div[2]/div[1]/div[2]/div[1]/div[10]/div[1]/div/a"}
  (Session info: chrome=133.0.6943.99)
Stacktrace:
	GetHandleVerifier [0x00A60283+25139]
	(No symbol) [0x009EB234]
	(No symbol) [0x008C04A3]
	(No symbol) [0x00908338]
	(No symbol) [0x0090868B]
	(No symbol) [0x00950F62]
	(No symbol) [0x0092CF84]
	(No symbol) [0x0094E6E1]
	(No symbol) [0x0092CD36]
	(No symbol) [0x008FBD29]
	(No symbol) [0x008FD064]
	GetHandleVerifier [0x00D6B143+3215603]
	GetHandleVerifier [0x00D822BA+3310186]
	GetHandleVerifier [0x00D7C4D2+3286146]
	GetHandleVerifier [0x00AF9C80+654384]
	(No symbol) [0x009F45BD]
	(No symbol) [0x009F14A8]
	(No symbol) [0x009F1647]
	(No symbol) [0x009E3D20]
	BaseThreadInitThunk [0x75BFFCC9+25]
	RtlGetAppContainerNamedObjectPath [0x77D982AE+286]
	RtlGetAppContainerNamedObjectPath [0x77D9827E+238]


In [138]:
data = {'장소': place_names, '주소': addresses, '전화번호': phones}
df = pd.DataFrame(data)

df

,장소,주소,전화번호
0,서정대리점 문학초점,"전북 전주시 완산구 유연로 154, (효자동3가,문학타워)",063-903-1166
1,하은대리점 본점,전북 익산시 무왕로 1171 (부송동),063-834-4599
2,한성대리점 최강점,"전북 익산시 궁동로 124 (부송동,춘영빌딩)",063-834-7011
3,PS&M (군산점),"전북 군산시 월명로 255, 105호(수송동,신한월드타워)",070-7470-9300
4,더 대리점 서신점,"전북 전주시 완산구 당산로 34, (서신동,비사벌아파트상가)",063-913-1020
...,...,...,...
79,한성대리점 나운2호점,전북 군산시 신설로 2 (나운동) SK텔레콤,063-910-5949
80,신세계대리점 고창점,"전북 고창군 고창읍 보릿골로 115, 1층",063-564-8017
81,PS&M (무주점),전북 무주군 무주읍 단천로 105 (),063-323-0032
82,PS&M (효자점),"전북 전주시 완산구 세내로 540 (효자동3가,전주프라자)",070-7470-2100


In [139]:
df.to_excel('SKT_전북.xlsx', index=False)

In [140]:
url = "https://www.lguplus.com/support/store-address"
driver.get(url)
time.sleep(3)
place_names = []
addresses = []
phones = []

In [141]:
element1 = driver.find_element(By.XPATH, '//*[@id="itemSido"]')
dropdown = Select(element1)
dropdown.select_by_value("전북특별자치도")
time.sleep(1)

element2 = driver.find_element(By.XPATH, '//*[@id="itemGungu"]')
dropdown2 = Select(element2)
dropdown2.select_by_visible_text("시/군/구 선택")
time.sleep(1)

# search_input = driver.find_element(By.XPATH, '//*[@id="_uid_187"]')
# search_input.send_keys('서울')
# time.sleep(1)

search_button = driver.find_element(By.XPATH, '//*[@id="_uid_188"]')
search_button.click()
time.sleep(3)

In [142]:
for page in tqdm(range(1, 59)):
        for i in range(2):
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
                time.sleep(3)
        for i in range(1, 10):
                title_button = driver.find_element(By.XPATH, f"/html/body/div[1]/div/div/div[4]/div[1]/div/div[2]/div/div[2]/div[1]/div[2]/div[1]/div[2]/div[1]/ul/li[{i}]/a")
                place_names.append(title_button.text)

                address_element = driver.find_element(By.XPATH, f'//*[@id="contentsSection"]/div/div[2]/div[1]/div[2]/div[1]/div[2]/div[1]/ul/li[{i}]/div[1]/p[1]')
                addresses.append(address_element.text)

#                 phone_element = driver.find_element(By.XPATH, f'//*[@id="store-map-layout"]/div[1]/div[{i}]/div[1]/div/div/div[4]').text
#                 phones.append(phone_element)

        if page < 58:

                for _ in range(2):
                        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_UP)
                        time.sleep(2)
                wait = WebDriverWait(driver, 10)
                next_button = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="contentsSection"]/div/div[2]/div[1]/div[2]/div[1]/div[2]/div[2]/ul/li[8]/button')))
                next_button.click()
                time.sleep(3)
                driver.execute_script("window.scrollTo(0, 0);")
                time.sleep(3)

 12%|██████████                                                                         | 7/58 [02:01<14:46, 17.37s/it]


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[1]/div/div/div[4]/div[1]/div/div[2]/div/div[2]/div[1]/div[2]/div[1]/div[2]/div[1]/ul/li[2]/a"}
  (Session info: chrome=133.0.6943.99)
Stacktrace:
	GetHandleVerifier [0x00A60283+25139]
	(No symbol) [0x009EB234]
	(No symbol) [0x008C04A3]
	(No symbol) [0x00908338]
	(No symbol) [0x0090868B]
	(No symbol) [0x00950F62]
	(No symbol) [0x0092CF84]
	(No symbol) [0x0094E6E1]
	(No symbol) [0x0092CD36]
	(No symbol) [0x008FBD29]
	(No symbol) [0x008FD064]
	GetHandleVerifier [0x00D6B143+3215603]
	GetHandleVerifier [0x00D822BA+3310186]
	GetHandleVerifier [0x00D7C4D2+3286146]
	GetHandleVerifier [0x00AF9C80+654384]
	(No symbol) [0x009F45BD]
	(No symbol) [0x009F14A8]
	(No symbol) [0x009F1647]
	(No symbol) [0x009E3D20]
	BaseThreadInitThunk [0x75BFFCC9+25]
	RtlGetAppContainerNamedObjectPath [0x77D982AE+286]
	RtlGetAppContainerNamedObjectPath [0x77D9827E+238]


In [143]:
data = {'장소': place_names, '주소': addresses}
df = pd.DataFrame(data)


In [144]:
df

,장소,주소
0,1\n조촌동_디오션G플레이스점,도로명\n전북 군산시 궁포안2길27 G플레이스 128호(조촌동)
1,2\n경암동_롯데몰점,"도로명\n전북 군산시 궁포1로 24-9 105호 (조촌동, 엔비빌딩)"
2,3\n문화동_현대코아점,도로명\n전북 군산시 백토로 108 현대코아사거리 미니스톱옆 LGU+군산문화직영점 ...
3,4\n산북동_동아아파트점,도로명\n전북 군산시 동아로 155 (산북동)
4,5\n수송동_제일아파트점,도로명\n전북 군산시 월명로 255 104호 (수송동)
...,...,...
59,6\n평화동_경복궁아파트점,"도로명\n전북 전주시 완산구 평화로 120 109호 (평화동2가, 평화동두산경복궁아파트)"
60,7\n평화동_현대아이파크점,도로명\n전북 전주시 완산구 소대배기로 20 1층(평화동2가)
61,8\n수성동_정읍CGV점,도로명\n전북 정읍시 중앙1길 1 (수성동)
62,9\n상동_정읍여고점,도로명\n전북 정읍시 신경2길 28 (상동)


In [145]:
df.to_excel('LGU+_전북.xlsx', index=False)

In [111]:
next_button = wait.until(EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/div/div/div[4]/div[1]/div/div[2]/div/div[2]/div[1]/div[2]/div[1]/div[2]/div[2]/ul/li[8]/button')))
next_button.click()

In [112]:
driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")

In [107]:
driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")

In [106]:
search_input.get_attribute("value")

'서울'

In [ ]:
/html/body/div[1]/div/div/div[4]/div[1]/div/div[2]/div/div[2]/div[1]/div[2]/div[1]/div[2]/div[1]/ul/li[1]

In [ ]:
/html/body/div[1]/div/div/div[4]/div[1]/div/div[2]/div/div[2]/div[1]/div[2]/div[1]/div[2]/div[1]/ul/li[1]/a

In [98]:
driver.find_element(By.XPATH, '//*[@id="_uid_187"]')

<selenium.webdriver.remote.webelement.WebElement (session="a9ad698a23643246095641e0c52c5878", element="f.91CDD13CEC21250EB73CE4FD7D667B92.d.895560FF38887640796C5D56A8012F32.e.1102")>

In [109]:
title_button.text

'1\n상계동_수락산역점'

In [113]:
address_element = driver.find_element(By.XPATH, f'//*[@id="contentsSection"]/div/div[2]/div[1]/div[2]/div[1]/div[2]/div[1]/ul/li[1]/div[1]/p[1]')
address_element.text

'도로명\n서울시 노원구 노해로81길 5-6 2층(상계동,노원스퀘어)'

In [150]:
lst = ['강원', '경기', '경남', '경북', '광주', '대구', '대전', '부산', '서울', '세종', '울산', '인천', '전남', '전북', '제주', '충남', '충북']

In [156]:
skt_df = pd.read_excel('SKT_강원.xlsx')

In [157]:
for loc in lst[1:]:
    file_name = f'SKT_{loc}.xlsx'
    temp_df = pd.read_excel(file_name)
    skt_df = pd.concat([skt_df, temp_df])
    

In [158]:
skt_df

,장소,주소,전화번호
0,원주이동통신대리점 KTI 점,강원 영월군 영월읍 하송로 48 (),033-372-6999
1,T SHOP대리점 강남8호점,"강원 춘천시 영서로2279번길 3, (온의동) 퐁당디저트",033-910-8881
2,유니온대리점 홍천본점,강원 홍천군 홍천읍 번영로 31-1번지,033-435-0011
3,인제중앙대리점 본점,강원 인제군 인제읍 비봉로 5-1 (),033-461-8282
4,온유대리점 본점,"강원 원주시 서원대로 159, 1층 (단계동)",0507-1496-0112
...,...,...,...
116,흥덕대리점 광혜원곰점,충북 진천군 광혜원면 중리2길 13-2 (광혜원리 427-10),043-537-3010
117,흥덕대리점 금릉점,"충북 충주시 번영대로 103, (연수동)",043-847-1140
118,흥덕대리점 봉명자이점,"충북 청주시 흥덕구 직지대로 578, 1층 101호(봉명동)",043-900-9555
119,흥덕대리점 옥산점,충북 청주시 흥덕구 옥산면 청주역로 650 (대일설비공사),043-263-5010


In [159]:
skt_df.to_excel('SKT매장정보.xlsx', index=False)

In [160]:
lg_df = pd.read_excel('LGU+_강원.xlsx')
lg_df

,장소,주소
0,1\n철원_동송점,"도로명\n강원도 철원군 동송읍 금학로 213 1층(이평리,야후 PC방) LG유플러스"
1,2\n중앙동_로데오점,도로명\n강원 속초시 중앙로 145-1 1층 (중앙동)
2,3\n조양동_이마트점,도로명\n강원도 속초시 동해대로 4024 1층 117호 ~118호(조양동)
3,4\n우두동_우두사거리점,도로명\n강원 춘천시 충열로 10-6 1층 LG 유플러스 (우두동)
4,5\n후평동_후평사거리점,도로명\n강원도 춘천시 춘천로 315 1동 1층(후평동)
5,6\n후평동_포스코점,도로명\n강원 춘천시 후석로326번길 5 106호 (후평동)
6,7\n일상의틈_조양동_춘천명동점,도로명\n강원도 춘천시 금강로 61 (조양동)
7,8\n약사명동_춘천중앙로점,도로명\n강원도 춘천시 중앙로 74 1층 1호(중앙로2가)
8,9\n후평동_지적공사점,도로명\n강원 춘천시 후석로 312 (후평동)
9,1\n운교동_팔호광장점,도로명\n강원 춘천시 춘천로 191 (운교동)


In [161]:
for loc in lst[1:]:
    file_name = f'LGU+_{loc}.xlsx'
    temp_df = pd.read_excel(file_name)
    lg_df = pd.concat([lg_df, temp_df])
    

In [169]:
lg_df['장소'] = lg_df['장소'].apply(lambda x:x[2:])

In [170]:
lg_df['주소'] = lg_df['주소'].apply(lambda x:x[4:])

In [163]:
lg_df.to_excel('LG매장정보(수정).xlsx', index=False)

In [168]:
lg_df = pd.read_excel('LG매장정보.xlsx')
lg_df

,장소,주소
0,1\n철원_동송점,"도로명\n강원도 철원군 동송읍 금학로 213 1층(이평리,야후 PC방) LG유플러스"
1,2\n중앙동_로데오점,도로명\n강원 속초시 중앙로 145-1 1층 (중앙동)
2,3\n조양동_이마트점,도로명\n강원도 속초시 동해대로 4024 1층 117호 ~118호(조양동)
3,4\n우두동_우두사거리점,도로명\n강원 춘천시 충열로 10-6 1층 LG 유플러스 (우두동)
4,5\n후평동_후평사거리점,도로명\n강원도 춘천시 춘천로 315 1동 1층(후평동)
...,...,...
1796,1\n용암동_농협사거리점,도로명\n충북 청주시 상당구 월평로 192 (용암동)
1797,2\n분평동_주민센터점,도로명\n충북 청주시 서원구 분평로 32 (분평동)
1798,3\n분평동_남부신협점,도로명\n충북 청주시 서원구 1순환로1107번길 33 (분평동)
1799,4\n옥천읍_중앙로점,"도로명\n충북 옥천군 옥천읍 중앙로 50 1층 LG유플러스(금구리,상점)"


In [172]:
len(lg_df)

1801

In [180]:
lg_df['일련번호'] = list(range(1, 1802))

In [176]:
lg_df['사업자'] = ['LGU+'] * 1801

In [178]:
lg_df['대리점명'] = lg_df['장소']

In [179]:
lg_df

,장소,주소,일려번호,사업자,대리점명
0,철원_동송점,"강원도 철원군 동송읍 금학로 213 1층(이평리,야후 PC방) LG유플러스",1,LGU+,철원_동송점
1,중앙동_로데오점,강원 속초시 중앙로 145-1 1층 (중앙동),2,LGU+,중앙동_로데오점
2,조양동_이마트점,강원도 속초시 동해대로 4024 1층 117호 ~118호(조양동),3,LGU+,조양동_이마트점
3,우두동_우두사거리점,강원 춘천시 충열로 10-6 1층 LG 유플러스 (우두동),4,LGU+,우두동_우두사거리점
4,후평동_후평사거리점,강원도 춘천시 춘천로 315 1동 1층(후평동),5,LGU+,후평동_후평사거리점
...,...,...,...,...,...
1796,용암동_농협사거리점,충북 청주시 상당구 월평로 192 (용암동),1797,LGU+,용암동_농협사거리점
1797,분평동_주민센터점,충북 청주시 서원구 분평로 32 (분평동),1798,LGU+,분평동_주민센터점
1798,분평동_남부신협점,충북 청주시 서원구 1순환로1107번길 33 (분평동),1799,LGU+,분평동_남부신협점
1799,옥천읍_중앙로점,"충북 옥천군 옥천읍 중앙로 50 1층 LG유플러스(금구리,상점)",1800,LGU+,옥천읍_중앙로점


In [181]:
lg_df = lg_df[['일련번호', '사업자', '대리점명', '주소']]

In [182]:
lg_df

,일련번호,사업자,대리점명,주소
0,1,LGU+,철원_동송점,"강원도 철원군 동송읍 금학로 213 1층(이평리,야후 PC방) LG유플러스"
1,2,LGU+,중앙동_로데오점,강원 속초시 중앙로 145-1 1층 (중앙동)
2,3,LGU+,조양동_이마트점,강원도 속초시 동해대로 4024 1층 117호 ~118호(조양동)
3,4,LGU+,우두동_우두사거리점,강원 춘천시 충열로 10-6 1층 LG 유플러스 (우두동)
4,5,LGU+,후평동_후평사거리점,강원도 춘천시 춘천로 315 1동 1층(후평동)
...,...,...,...,...
1796,1797,LGU+,용암동_농협사거리점,충북 청주시 상당구 월평로 192 (용암동)
1797,1798,LGU+,분평동_주민센터점,충북 청주시 서원구 분평로 32 (분평동)
1798,1799,LGU+,분평동_남부신협점,충북 청주시 서원구 1순환로1107번길 33 (분평동)
1799,1800,LGU+,옥천읍_중앙로점,"충북 옥천군 옥천읍 중앙로 50 1층 LG유플러스(금구리,상점)"


In [183]:
skt_df

,장소,주소,전화번호
0,원주이동통신대리점 KTI 점,강원 영월군 영월읍 하송로 48 (),033-372-6999
1,T SHOP대리점 강남8호점,"강원 춘천시 영서로2279번길 3, (온의동) 퐁당디저트",033-910-8881
2,유니온대리점 홍천본점,강원 홍천군 홍천읍 번영로 31-1번지,033-435-0011
3,인제중앙대리점 본점,강원 인제군 인제읍 비봉로 5-1 (),033-461-8282
4,온유대리점 본점,"강원 원주시 서원대로 159, 1층 (단계동)",0507-1496-0112
...,...,...,...
116,흥덕대리점 광혜원곰점,충북 진천군 광혜원면 중리2길 13-2 (광혜원리 427-10),043-537-3010
117,흥덕대리점 금릉점,"충북 충주시 번영대로 103, (연수동)",043-847-1140
118,흥덕대리점 봉명자이점,"충북 청주시 흥덕구 직지대로 578, 1층 101호(봉명동)",043-900-9555
119,흥덕대리점 옥산점,충북 청주시 흥덕구 옥산면 청주역로 650 (대일설비공사),043-263-5010


In [184]:
skt_df['일련번호'] = list(range(1, 2746))

In [185]:
skt_df['사업자'] = ['SKT'] * 2745

In [187]:
skt_df['대리점명'] = skt_df['장소']

In [188]:
skt_df = skt_df[['일련번호', '사업자', '대리점명', '주소']]

In [189]:
skt_df

,일련번호,사업자,대리점명,주소
0,1,SKT,원주이동통신대리점 KTI 점,강원 영월군 영월읍 하송로 48 ()
1,2,SKT,T SHOP대리점 강남8호점,"강원 춘천시 영서로2279번길 3, (온의동) 퐁당디저트"
2,3,SKT,유니온대리점 홍천본점,강원 홍천군 홍천읍 번영로 31-1번지
3,4,SKT,인제중앙대리점 본점,강원 인제군 인제읍 비봉로 5-1 ()
4,5,SKT,온유대리점 본점,"강원 원주시 서원대로 159, 1층 (단계동)"
...,...,...,...,...
116,2741,SKT,흥덕대리점 광혜원곰점,충북 진천군 광혜원면 중리2길 13-2 (광혜원리 427-10)
117,2742,SKT,흥덕대리점 금릉점,"충북 충주시 번영대로 103, (연수동)"
118,2743,SKT,흥덕대리점 봉명자이점,"충북 청주시 흥덕구 직지대로 578, 1층 101호(봉명동)"
119,2744,SKT,흥덕대리점 옥산점,충북 청주시 흥덕구 옥산면 청주역로 650 (대일설비공사)


In [190]:
total_df = pd.concat([skt_df, lg_df])
total_df

,일련번호,사업자,대리점명,주소
0,1,SKT,원주이동통신대리점 KTI 점,강원 영월군 영월읍 하송로 48 ()
1,2,SKT,T SHOP대리점 강남8호점,"강원 춘천시 영서로2279번길 3, (온의동) 퐁당디저트"
2,3,SKT,유니온대리점 홍천본점,강원 홍천군 홍천읍 번영로 31-1번지
3,4,SKT,인제중앙대리점 본점,강원 인제군 인제읍 비봉로 5-1 ()
4,5,SKT,온유대리점 본점,"강원 원주시 서원대로 159, 1층 (단계동)"
...,...,...,...,...
1796,1797,LGU+,용암동_농협사거리점,충북 청주시 상당구 월평로 192 (용암동)
1797,1798,LGU+,분평동_주민센터점,충북 청주시 서원구 분평로 32 (분평동)
1798,1799,LGU+,분평동_남부신협점,충북 청주시 서원구 1순환로1107번길 33 (분평동)
1799,1800,LGU+,옥천읍_중앙로점,"충북 옥천군 옥천읍 중앙로 50 1층 LG유플러스(금구리,상점)"


In [191]:
total_df['일련번호'] = list(range(1, 4547))

In [193]:
total_df.to_excel('타사매장.xlsx')